In [ ]:
import argparse
import os
import random
import torch
import torch.nn as nn
import numpy as np
from utils import *
from model import *
from torch.utils.data import TensorDataset
from sklearn.metrics import recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
class Args:
    def __init__(self):
        self.config = 'default'
        self.dataset = 'HHAR'
        self.model = 'UniTS'
        self.log = 'log'
        self.exp = ''
        self.ratio = 0.2
        self.epochs = 50
        self.lr = 1e-3
        self.batch_size = 64
        self.save = True #'BCE'
        self.test_only = False
        self.input_size = 100 # 24 #256    
        self.input_channel = 6 #113 #45
        self.hheads = 9
        self.SENSOR_AXIS = 3
        self.corr='Gaussian' # Gaussian, ConsecutiveZeros, Both, MissingNode
        # self.sigma = [0.1, 20, 50]
        self.sigma = [0.1]
        self.num_labels=6
        self.n_gpu = 0
        self.random_seed = 0
        
args = Args()

args.log_path = os.path.join(args.log, args.dataset)
if not os.path.exists(args.log_path):
    os.mkdir(args.log_path)
args.model_save_path = os.path.join(args.log_path, args.model + '_'+ args.config +'_'+args.corr+'_'+str(args.sigma[0])+'_' + str(args.random_seed) + '.pt')
config = read_config(args.config + '.yaml')

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(args.n_gpu) 

torch.cuda.set_device(0)
device = torch.device(0 if torch.cuda.is_available() else "cpu")


In [ ]:
log = set_up_logging(args, config)
args.log = log

## Our pre-processing

In [ ]:
act_labels_txt = ['bike', 'sit', 'std', 'walk', 'stair_up','stair_dwn']

In [ ]:
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

In [ ]:
folder_path = '../../../../Dataset/'
X_total = np.load(folder_path+'hhar_time_X.npy').astype('float32')
y_total = np.load(folder_path+'hhar_time_y.npy').astype(int)

X_total=np.nan_to_num(X_total)
for i in range(X_total.shape[1]):
    ch_data = X_total[:,i,:] # the data of channel id
    scaler = MinMaxScaler()
    ch_data = scaler.fit_transform(ch_data) # scale the data in this channel to [0,1]
    X_total[:,i,:] = ch_data # assign normalized data to normalized_X    
    
X_total = np.transpose(X_total, (0,2,1))
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=args.random_seed)

ytrain = np.argmax(y_train, axis=1)
ytest = np.argmax(y_test, axis=1)

In [ ]:
print(config.window_list)
print(config.stride_list)
print(config.k_list)
print(config.layer_num)
print(config.hidden_channel)
# Adjust model parameters based on our preprocessing
config.window_list = [7, 16, 32, 48, 64, 80, 96]
config.stride_list = [3, 8, 16, 24, 32, 40, 48]
config.k_list = [3, 8, 16, 24, 24, 32, 32]

In [ ]:
if args.model == 'UniTS':
    model = UniTS(input_size = args.input_size, sensor_num = args.input_channel, layer_num = config.layer_num,
    window_list = config.window_list, stride_list = config.stride_list, k_list = config.k_list,
    out_dim = args.num_labels, hidden_channel = config.hidden_channel).cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)


total_params = sum(p.numel() for p in model.parameters())
log('Total parameters: ' + str(total_params))

if args.test_only:
    if os.path.exists(args.model_save_path):
        model.load_state_dict(torch.load(args.model_save_path))
        test(model, xtest, ytest)
    else:
        log("Model state dict not found!")

In [ ]:
def corrupt(args, x):
    if args.corr=='Gaussian':
        noise = args.sigma[0] * torch.randn(x.size()).type_as(x)
        return x + noise
    # lambdas reuse the sigma variable, unpack
    if args.corr == 'ConsecutiveZeros':
        lambda_corr = args.sigma[0] # lambda for missing data period
        lambda_norm = args.sigma[1] # lambda for normal data periodß

        mask = torch.ones_like(x)
        for sample_id in range(mask.shape[0]):
            for ch_id in range(mask.shape[2]):  
                ptr = 0
                is_corrupted = False
                while ptr < mask.shape[1]:
                    if is_corrupted:
                        corr_duration = int(np.random.exponential(scale=lambda_corr))
                        mask[sample_id, ptr:min(mask.shape[1], ptr + corr_duration), ch_id] = 0
                        ptr = min(mask.shape[1], ptr + corr_duration)
                        is_corrupted = False
                    else:
                        norm_duration = int(np.random.exponential(scale=lambda_norm))
                        ptr = min(mask.shape[1], ptr + norm_duration)
                        is_corrupted = True
        x = torch.mul(x, mask)
        return x
    if args.corr == 'Both':
        noise = args.sigma[0] * torch.randn(x.size()).type_as(x)
        x = x + noise
        # lambdas reuse the sigma variable, unpack
        lambda_corr = args.sigma[1] # lambda for missing data period
        lambda_norm = args.sigma[2] # lambda for normal data periodß

        mask = torch.ones_like(x)
        for sample_id in range(mask.shape[0]):
            for ch_id in range(mask.shape[2]):  
                ptr = 0
                is_corrupted = False
                while ptr < mask.shape[1]:
                    if is_corrupted:
                        corr_duration = int(np.random.exponential(scale=lambda_corr))
                        mask[sample_id, ptr:min(mask.shape[1], ptr + corr_duration), ch_id] = 0
                        ptr = min(mask.shape[1], ptr + corr_duration)
                        is_corrupted = False
                    else:
                        norm_duration = int(np.random.exponential(scale=lambda_norm))
                        ptr = min(mask.shape[1], ptr + norm_duration)
                        is_corrupted = True
        x = torch.mul(x, mask)
        return x
    
    elif args.corr == 'MissingNode':
        # lambdas reuse the sigma variable, unpack
        if args.dataset == 'PAMAP2':
            imu_chs = [[0,1,2, 3,4,5, 6,7,8], [9,10,11, 12,13,14, 15,16,17], [18,19,20, 21,22,23, 24,25,26]] # [imu1, imu2, imu3]
        elif args.dataset == 'OPPO':
            imu_chs = [
                [0,1,2], [3,4,5], [6,7,8], [9,10,11], [12,13,14], [15,16,17], [18,19,20], [21,22,23], [24,25,26], [27,28,29], [30,31,32], [33,34,35], # 12 accs
                [36,37,38, 39,40,41, 42,43,44], [45,46,47, 48,49,50, 51,52,53], [54,55,56, 57,58,59, 60,61,62], [63,64,65, 66,67,68, 69,70,71], [72,73,74, 75,76,77, 78,79,80], # 5 IMUs
                [81,82,83, 84,85,86, 87,88,89, 90,91,92, 93,94,95, 96,], [97,98,99, 100,101,102, 103,104,105, 106,107,108, 109,110,111, 112] # 2 shoe sensors
            ]
        elif args.dataset == 'HHAR':
            imu_chs = [[0,1,2], [3,4,5]] # [imu1, imu2, imu3]

        lambda_corr = args.sigma[0] # lambda for missing data period
        lambda_norm = args.sigma[1] # lambda for normal data periodß

        mask = torch.ones_like(x)
        for sample_id in range(mask.shape[0]):
            if args.dataset == 'PAMAP2':
                node_num = 3
            elif args.dataset == 'OPPO':
                node_num = 19 # 5+12+2
            elif args.dataset == 'HHAR':
                node_num = 2

            for imu_id in range(node_num):  
                ptr = 0
                is_corrupted = False
                while ptr < mask.shape[1]:
                    if is_corrupted:
                        corr_duration = int(np.random.exponential(scale=lambda_corr))
                        mask[sample_id, ptr:min(mask.shape[1], ptr + corr_duration), imu_chs[imu_id]] = 0
                        ptr = min(mask.shape[1], ptr + corr_duration)
                        is_corrupted = False
                    else:
                        norm_duration = int(np.random.exponential(scale=lambda_norm))
                        ptr = min(mask.shape[1], ptr + norm_duration)
                        is_corrupted = True
        x = torch.mul(x, mask)
        return x        
    
    else:
        print("Not implemented")

In [ ]:
xtrain = torch.from_numpy(X_train)
xtest = torch.from_numpy(X_test)
ytrain = torch.from_numpy(ytrain)
ytest = torch.from_numpy(ytest)

In [ ]:
xtrain = corrupt(args, xtrain)
xtest = corrupt(args, xtest)

In [ ]:
train_dataset = TensorDataset(xtrain, ytrain)
train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=args.batch_size, shuffle=True, drop_last = True)  

test_dataset = TensorDataset(xtest, ytest)
test_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=args.batch_size, shuffle=False, drop_last = True) 

In [ ]:
def test(model, test_loader):

    y_pred = []
    y_true = []
    with torch.no_grad():
        model.eval()
        for batch in test_loader:
            x, y = batch

            x = x.to(device)
            y_true += list(y.numpy())

            out = model(x)
            pred = torch.argmax(out, dim = -1)
            y_pred += pred.cpu().tolist()

    log("Accuracy:\n" + str(accuracy_score(y_true, y_pred)) +
        "\nWeighted F1:\n" + str(f1_score(y_true, y_pred, labels=list(range(args.num_labels)),average='weighted')) )

In [ ]:
loss_func = nn.CrossEntropyLoss()
try:
    for ep in range(1, 1+args.epochs):
        model.train()
        epoch_loss = 0
        log("Training epoch : " + str(ep))
        
        for batch in train_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)          
                 
            out = model(x)
            loss = loss_func(out, y)
            epoch_loss += loss.cpu().item()

            optimizer.zero_grad()           
            loss.backward()
            optimizer.step()

        log("Training loss : " + str(epoch_loss / (len(xtrain) / args.batch_size)))
        test(model, test_loader)
        log("----------------------------")


except KeyboardInterrupt:
    print('Exiting from training early')
    test(model, test_loader)
# if args.save:
#     torch.save(model.state_dict(), args.model_save_path)

In [ ]:
torch.save(model.state_dict(), args.model_save_path)

In [ ]:
class eval_Args:
    def __init__(self):
        self.corr=args.corr
        self.sigma=[0.05]
        self.dataset=args.dataset
eval_args = eval_Args()

xtest_eval = torch.from_numpy(X_test)
xtest_eval = corrupt(eval_args, xtest_eval)

test_dataset_eval = TensorDataset(xtest_eval, ytest)
test_loader_eval = torch.utils.data.DataLoader(test_dataset_eval,
    batch_size=args.batch_size, shuffle=False, drop_last = True) 

test(model, test_loader_eval)